# Benchmark of existing approaches for detecting machine-generated text

## Solaiman

_Irene Solaiman, Miles Brundage, Jack Clark, Amanda Askell, Ariel Herbert-Voss, Jeff Wu, Alec Radford,
Gretchen Krueger, Jong Wook Kim, Sarah Kreps, Miles McCain, Alex Newhouse, Jason Blazakis, Kris McGuffie, and Jasmine Wang. 2019. Release strategies and the social impacts of language models._

In [1]:
import os
import json
import numpy as np
from scipy import sparse

from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
def _load_split(data_dir, source, split, n=np.inf):
    path = os.path.join(data_dir, f'{source}.{split}.jsonl')
    texts = []
    for i, line in enumerate(open(path)):
        if i >= n:
            break
        texts.append(json.loads(line)['text'])
    return texts

def load_split(data_dir, source, split, n=np.inf):
    webtext = _load_split(data_dir, 'webtext-clean', split, n=n//2)
    gen = _load_split(data_dir, source, split, n=n//2)
    texts = webtext+gen
    labels = [0]*len(webtext)+[1]*len(gen)
    return texts, labels

In [48]:
data_dir = './../../get_text_detect_space/datasets/GPT2vsWebText/'
log_dir = './results/solaiman' 
source='xl-1542M-clean'
n_train=200000
n_valid=10000
n_jobs=-1
verbose=False

In [49]:
train_texts, train_labels = load_split(data_dir, source, 'train', n=n_train)
valid_texts, valid_labels = load_split(data_dir, source, 'valid', n=n_valid)
test_texts, test_labels = load_split(data_dir, source, 'test')

In [50]:
len(train_texts)

200000

In [51]:
len(test_texts)

10000

In [52]:
vect = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_features=2**21)
train_features = vect.fit_transform(train_texts)

In [53]:
valid_features = vect.transform(valid_texts)
test_features = vect.transform(test_texts)

In [54]:
model = LogisticRegression(solver='liblinear')

skip the following grid search grid and use the default params

In [ ]:
# params = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
params = {'C': [1/8, 1, 8]}
split = PredefinedSplit([-1]*n_train+[0]*n_valid)
search = GridSearchCV(model, params, cv=split, n_jobs=n_jobs, verbose=verbose, refit=False)
search.fit(sparse.vstack([train_features, valid_features]), train_labels+valid_labels)
model = model.set_params(**search.best_params_)

In [55]:
model.fit(train_features, train_labels)

LogisticRegression(solver='liblinear')

In [56]:
import pickle
filename = 'solaiman_logreg_linear_xl-1542M-clean_webtext-clean.sav'
pickle.dump(model, open(filename, 'wb'))

In [57]:
valid_accuracy = model.score(valid_features, valid_labels)*100.
test_accuracy = model.score(test_features, test_labels)*100.

In [59]:
data = {
    'source':source,
    'n_train':n_train,
    'valid_accuracy':valid_accuracy,
    'test_accuracy':test_accuracy
}
print(data)

{'source': 'xl-1542M-clean', 'n_train': 200000, 'valid_accuracy': 68.77, 'test_accuracy': 69.19999999999999}


In [60]:
json.dump(data, open(os.path.join(log_dir, f'{source}.json'), 'w'))

In [61]:
source='xl-1542M'
train_texts, train_labels = load_split(data_dir, source, 'train', n=n_train)
valid_texts, valid_labels = load_split(data_dir, source, 'valid', n=n_valid)
test_texts, test_labels = load_split(data_dir, source, 'test')

In [62]:
vect = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_features=2**21)
train_features = vect.fit_transform(train_texts)
valid_features = vect.transform(valid_texts)
test_features = vect.transform(test_texts)

In [63]:
model.fit(train_features, train_labels)

LogisticRegression(solver='liblinear')

In [64]:
filename = 'solaiman_logreg_linear_xl-1542M.sav'
pickle.dump(model, open(filename, 'wb'))

In [65]:
valid_accuracy = model.score(valid_features, valid_labels)*100.
test_accuracy = model.score(test_features, test_labels)*100.

In [66]:
data = {
    'source':source,
    'n_train':n_train,
    'valid_accuracy':valid_accuracy,
    'test_accuracy':test_accuracy
}

In [67]:
data

{'source': 'xl-1542M',
 'n_train': 200000,
 'valid_accuracy': 57.50999999999999,
 'test_accuracy': 58.42}

In [68]:
json.dump(data, open(os.path.join(log_dir, f'{source}.json'), 'w'))